In [ ]:
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

In [3]:
def analysis_one(df):
    grup_vendor = (df.groupby('vendor_id')['passenger_count']
            .sum()
            .to_frame()
            .reset_index())

    grup_km = (df.groupby('vendor_id')['trip_distance_km']
            .sum()
            .to_frame()
            .reset_index())

    grup_pickup = (df.groupby(['pickup_hour', 'vendor_id'])['passenger_count']
            .sum()
            .to_frame()
            .reset_index())

    grup_dropoff = (df.groupby(['dropoff_hour', 'vendor_id'])['passenger_count']
            .sum()
            .to_frame()
            .reset_index())

    grup_month_name = (df.groupby(['month_name', 'vendor_id'])['passenger_count']
            .sum()
            .to_frame()
            .reset_index())
    grup_month_name["month_name"] = grup_month_name["month_name"].map({"JAN":1,"FEB":2,"MAR":3,
                                                                    "APR":4,"MAY":5,"JUN":6})
    grup_month_name = grup_month_name.sort_values(by=['month_name'])
    grup_month_name["month_name"] = grup_month_name["month_name"].map({1:"JAN",2:"FEB",3:"MAR",
                                                                    4:"APR",5:"MAY",6:"JUN"})
    
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

    fig.add_trace(go.Pie(values = [grup_vendor['passenger_count'][0], grup_vendor['passenger_count'][1]],
                name="Total passageiros",
                labels = ["Categória de Táxi 1" , "Categória de Táxi 2"],
                marker = dict(colors = ['royalblue' ,'red'], line = dict(color = "black",
                                                                            width =  1.5)),
                rotation  = 90,
                pull=[0, 0.1],
                #hole=.4
                #hoverinfo = "label+percent",
                ), row=1, col=1
            )

    fig.add_trace(go.Pie(values = [grup_km['trip_distance_km'][0], grup_km['trip_distance_km'][1]],
                name="Total KM",
                labels = ["Categória de Táxi 1" , "Categória de Táxi 2"],
                marker = dict(colors = ['royalblue' ,'red'], line = dict(color = "black",
                                                                            width =  1.5)),
                rotation  = 90,
                pull=[0, 0.1],
                #hole=.4
                #hoverinfo = "label+percent",
                ), row=1, col=2
            
            )

    fig.update_layout(dict(title = "Total de passageiros e quilomêtros por categória de Táxi", title_x=0.5,
                            legend_title_text = "Categória",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            #height=600, width=800,
                        )
                    )

    def grup_month(category, df):

        if category == 'class_one':
            nome = 'Categória de Táxi 1'
        elif category == 'class_two':
            nome = 'Categória de Táxi 2'
        
        tracer = go.Bar(x=df[(df['vendor_id'] == category)]['month_name'], 
                        y=df[(df['vendor_id'] == category)]['passenger_count'],
                        text=list(df[(df['vendor_id'] == category)]['passenger_count']), 
                        textposition='auto',
                        name = nome, marker = dict(line = dict(color = "black",
                                                                    width =  1.5))
                        )
        return tracer

    def layout_plot_month(title) :
        layout = go.Layout(dict(title = title,  title_x=0.5,
                            legend_title_text = "Categória",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                        title = "Meses",
                                        zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                        title = "Quantidade de passageiros",
                                        zerolinewidth=1,ticklen=5,gridwidth=2),
                            margin = dict(b = 100)
                            )
                        )
        return layout


    def grup_pickup_dropoff_hour(category, pickup_dropoff_hour, df):

        if category == 'class_one' and pickup_dropoff_hour == 'pickup_hour':
            nomeLabel = 'Pickup Hour Class One'
        elif category == 'class_two' and pickup_dropoff_hour == 'pickup_hour':
            nomeLabel = 'Pickup Hour Class Two'
        elif category == 'class_one' and pickup_dropoff_hour == 'dropoff_hour':
            nomeLabel = 'Dropoff Hour Class One'
        elif category == 'class_two' and pickup_dropoff_hour == 'dropoff_hour':
            nomeLabel = 'Dropoff Hour Class Two'

        tracer = go.Bar(x=df[df['vendor_id'] == category][pickup_dropoff_hour].map(str), 
                            y=df[df['vendor_id'] == category]['passenger_count'],
                            text=list(df[df['vendor_id'] == category]['passenger_count']), 
                            textposition='auto',
                            name = nomeLabel, marker = dict(line = dict(color = "black",
                                                                        width =  1.5))
                            )
        return tracer

    def layout_plot(title):
        layout = go.Layout(dict(title = title,  title_x=0.5,
                                legend_title_text = "Categória de Táxi",
                                plot_bgcolor  = "rgb(243,243,243)",
                                paper_bgcolor = "rgb(243,243,243)",
                                xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                            title = "Horário da corrida",
                                            zerolinewidth=1,ticklen=5,gridwidth=2),
                                yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                            title = "Quantidade de passageiros",
                                            zerolinewidth=1,ticklen=5,gridwidth=2),
                                margin = dict(b = 100)
                            )
                        )
        return layout

    #figure 1
    fig = go.Figure(data=fig)
    figigure_one = py.iplot(fig)

    #figure 2
    data1 = [grup_month('class_one', grup_month_name), grup_month('class_two', grup_month_name)]
    layout1  = layout_plot_month("Total de passageiros por meses separados por classe de Táxi")
    fig1 = go.Figure(data = data1,layout = layout1)
    figure_two = py.iplot(fig1)

    #figure 3
    data2 = [grup_pickup_dropoff_hour('class_one', 'pickup_hour',grup_pickup), grup_pickup_dropoff_hour('class_two', 'pickup_hour',grup_pickup),
            grup_pickup_dropoff_hour('class_one', 'dropoff_hour',grup_dropoff), grup_pickup_dropoff_hour('class_two', 'dropoff_hour',grup_dropoff)]
    layout2 = layout_plot("Total de passageiros por Embarque e Desembarque separados por classe de Táxi")
    fig2 = go.Figure(data=data2,layout=layout2)
    figure_tree = py.iplot(fig2)

    return figigure_one, figure_two, figure_tree

### Funções para os gráficos de categória de Táxi 2

In [ ]:
def analysis_one_ct(df):
    grup_pickup_week = (df.groupby(['pickup_hour', 'vendor_id','pickup_day_of_week'])['passenger_count']
            .sum()
            .to_frame()
            .reset_index())
    grup_dropoff_week = (df.groupby(['dropoff_hour', 'vendor_id','pickup_day_of_week'])['passenger_count']
            .sum()
            .to_frame()
            .reset_index())
    cont_pass = (df[(df['vendor_id'] == 'class_two')]['passenger_count']
            .value_counts()
            .rename_axis('quant_pass')
            .to_frame()
            .reset_index()
            .sort_values(by='quant_pass'))
    grup_km = (df.groupby(['vendor_id','month_name'])['trip_distance_km']
            .sum()
            .to_frame()
            .reset_index())
    grup_km = grup_km[(grup_km['vendor_id'] == 'class_two')]
    grup_km["month_name"] = grup_km["month_name"].map({"JAN":1,"FEB":2,"MAR":3,
                                                    "APR":4,"MAY":5,"JUN":6})
    grup_km = grup_km.sort_values(by=['month_name'])
    grup_km["month_name"] = grup_km["month_name"].map({1:"JAN",2:"FEB",3:"MAR",
                                                    4:"APR",5:"MAY",6:"JUN"})
    round_km = round(grup_km['trip_distance_km'],0)
    grup_km['trip_distance_km'] = round_km


    def cont_pass_valeu(df):
        cont = go.Scatter(x=df['quant_pass'].map(str), 
                        y=df['passenger_count'],
                        mode = "lines+markers",
                        marker = dict(color = "royalblue",line = dict(width =1)) 
                        )
        return cont

    def grup_pickup_dropoff_week(day, category, df, pickup_dropoff_hour):
        nome = day
        
        tracer = go.Bar(x=df[(df['vendor_id'] == category) & (df['pickup_day_of_week'] == day)][pickup_dropoff_hour].map(str), 
                        y=df[(df['vendor_id'] == category) & (df['pickup_day_of_week'] == day)]['passenger_count'],
                        text=list(df[(df['vendor_id'] == category) & (df['pickup_day_of_week'] == day)]['passenger_count']), 
                        textposition='auto',
                        name = nome, marker = dict(line = dict(color = "black",
                                                                    width =  1.5))
                        )
        return tracer

    def grup_KM(month, distance, df):
        
        tracer = go.Bar(x=df[month], 
                        y=df[distance],
                        text=list(df[distance]),
                        textposition='auto',
                        marker = dict(line = dict(color = "black",
                                                width =  1.5))
                        )
        return tracer

    #layout
    def layout_plot(title, legend_title, xaxis, yaxis) :
        layout = go.Layout(dict(title = title,  title_x=0.5,
                            legend_title_text = legend_title,
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                        title = xaxis,
                                        zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                        title = yaxis,
                                        zerolinewidth=1,ticklen=5,gridwidth=2),
                                margin = dict(b = 100)
                            )
                        )
        return layout

    #figure 1
    data    = cont_pass_valeu(cont_pass)
    layout  = layout_plot('Total de passageiros por viagem categória de Táxi 2', None, 'Numéros de passageiros', 'Quantidade de viagens')
    fig = go.Figure(data = data,layout = layout)
    figure_one = py.iplot(fig)

    #figure 2
    data1 = [grup_pickup_dropoff_week('Monday', 'class_two', grup_pickup_week, 'pickup_hour'), grup_pickup_dropoff_week('Tuesday', 'class_two', grup_pickup_week, 'pickup_hour'), 
            grup_pickup_dropoff_week('Wednesday', 'class_two', grup_pickup_week, 'pickup_hour'), grup_pickup_dropoff_week('Thursday', 'class_two', grup_pickup_week, 'pickup_hour'),
            grup_pickup_dropoff_week('Friday', 'class_two', grup_pickup_week, 'pickup_hour'), grup_pickup_dropoff_week('Saturday', 'class_two', grup_pickup_week, 'pickup_hour'),
            grup_pickup_dropoff_week('Sunday', 'class_two', grup_pickup_week, 'pickup_hour')]
    layout1  = layout_plot('Total de passageiros por embarque categória de Táxi 2', 'Dias da Semana', 'Horário da corrida', 'Quantidade de passageiros')
    fig1 = go.Figure(data = data1,layout = layout1)
    figure_two = py.iplot(fig1)

    #figure 3
    data2 = [grup_pickup_dropoff_week('Monday', 'class_two', grup_dropoff_week, 'dropoff_hour'), grup_pickup_dropoff_week('Tuesday', 'class_two', grup_dropoff_week, 'dropoff_hour'), 
            grup_pickup_dropoff_week('Wednesday', 'class_two', grup_dropoff_week, 'dropoff_hour'), grup_pickup_dropoff_week('Thursday', 'class_two', grup_dropoff_week, 'dropoff_hour'),
            grup_pickup_dropoff_week('Friday', 'class_two', grup_dropoff_week, 'dropoff_hour'), grup_pickup_dropoff_week('Saturday', 'class_two', grup_dropoff_week, 'dropoff_hour'),
            grup_pickup_dropoff_week('Sunday', 'class_two', grup_dropoff_week, 'dropoff_hour')]
    layout2  = layout_plot('Total de passegeiros por desembarque categória de Táxi 2', 'Dias da Semana', 'Horário da corrida', 'Quantidade de passageiros')
    fig2 = go.Figure(data = data2,layout = layout2)
    figure_tree = py.iplot(fig2)

    #figure 5
    data4    = grup_KM('month_name', 'trip_distance_km', grup_km)
    layout4  = layout_plot('Total de quilômetros percoridos por meses categória de Táxi 2', None, 'Meses', 'Total de Quilômetros')
    fig4 = go.Figure(data = data4,layout = layout4)
    figure_five = py.iplot(fig4)

    return figure_one, figure_two, figure_tree, figure_five

In [ ]:
def analysis_two_ct(df):

    JAN = ((df[(df['vendor_id'] == 'class_two') & (df['month_name'] == 'JAN') & (df['pickup_date'] >= '2016-01-01') & (df['pickup_date'] <= '2016-01-15')]['passenger_count']))
    FEB = ((df[(df['vendor_id'] == 'class_two') & (df['month_name'] == 'FEB') & (df['pickup_date'] >= '2016-02-01') & (df['pickup_date'] <= '2016-02-15')]['passenger_count']))
    MAR = ((df[(df['vendor_id'] == 'class_two') & (df['month_name'] == 'MAR') & (df['pickup_date'] >= '2016-03-01') & (df['pickup_date'] <= '2016-03-15')]['passenger_count']))
    APR = ((df[(df['vendor_id'] == 'class_two') & (df['month_name'] == 'APR') & (df['pickup_date'] >= '2016-04-01') & (df['pickup_date'] <= '2016-04-15')]['passenger_count']))
    MAY = ((df[(df['vendor_id'] == 'class_two') & (df['month_name'] == 'MAY') & (df['pickup_date'] >= '2016-05-01') & (df['pickup_date'] <= '2016-05-15')]['passenger_count']))
    JUN = ((df[(df['vendor_id'] == 'class_two') & (df['month_name'] == 'JUN') & (df['pickup_date'] >= '2016-06-01') & (df['pickup_date'] <= '2016-06-15')]['passenger_count']))

    x_data = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']

    y0 = JAN
    y1 = FEB
    y2 = MAR
    y3 = APR
    y4 = MAY
    y5 = JUN

    y_data = [y0, y1, y2, y3, y4, y5]

    colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
            'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']

    fig3 = go.Figure()

    for xd, yd, cls in zip(x_data, y_data, colors):
            fig3.add_trace(go.Box(
                y=yd,
                name=xd,
                boxpoints='all',
                jitter=0.5,
                whiskerwidth=0.2,
                fillcolor=cls,
                marker_size=2,
                line_width=1)
            )

    fig3 = fig3.update_layout(
        title='Distribuição de passageiros por meses com range de 0 a 15 dias categória de Táxi 2', title_x = 0.5,
        yaxis=dict(
            autorange=True,
            showgrid=True,
            zeroline=True,
            dtick=5,
            gridcolor='rgb(255, 255, 255)',
            gridwidth=1,
            zerolinecolor='rgb(255, 255, 255)',
            zerolinewidth=2,
        ),
        margin=dict(
            l=40,
            r=30,
            b=80,
            t=100,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )

    #figure 4
    figure_four = py.iplot(fig3)

    return figure_four

In [ ]:
def analysis_tree_ct(df):

    grup_lat_lon_pic = (df.groupby(['vendor_id', 'month_name','pickup_latitude', 'pickup_longitude'])['passenger_count']
        .sum()
        .to_frame()
        .reset_index())
    
    fig5 = px.scatter_mapbox(grup_lat_lon_pic[(grup_lat_lon_pic['vendor_id'] == 'class_two')], lat="pickup_latitude", lon="pickup_longitude", color="month_name",
                        size="passenger_count", color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=11, zoom=8, mapbox_style="carto-positron")
    figure_five = fig5.update_layout(legend_title_text = "Meses", title = "Distribuição de passageiros no Embarque por categória de Táxi 2", title_x=0.5)

    return figure_five

In [ ]:
def analysis_four_ct(df):

    grup_lat_lon_drop = (df.groupby(['vendor_id', 'month_name','dropoff_latitude', 'dropoff_longitude'])['passenger_count']
        .sum()
        .to_frame()
        .reset_index())
    
    fig6 = px.scatter_mapbox(grup_lat_lon_drop[(grup_lat_lon_drop['vendor_id'] == 'class_two')], lat="dropoff_latitude", lon="dropoff_longitude", color="month_name",
                        size="passenger_count", color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=11, zoom=8, mapbox_style="carto-positron")
    figure_six = fig6.update_layout(legend_title_text = "Meses", title = "Distribuição de passageiros no Desembarque por categória de Táxi 2", title_x=0.5)

    return figure_six

---

### Funções para os gráficos de categória de Táxi 1

In [ ]:
def analysis_one_co(df):
    grup_pickup_week = (df.groupby(['pickup_hour', 'vendor_id','pickup_day_of_week'])['passenger_count']
            .sum()
            .to_frame()
            .reset_index())
    grup_dropoff_week = (df.groupby(['dropoff_hour', 'vendor_id','pickup_day_of_week'])['passenger_count']
            .sum()
            .to_frame()
            .reset_index())
    cont_pass = (df[(df['vendor_id'] == 'class_one')]['passenger_count']
            .value_counts()
            .rename_axis('quant_pass')
            .to_frame()
            .reset_index()
            .sort_values(by='quant_pass'))
    grup_km = (df.groupby(['vendor_id','month_name'])['trip_distance_km']
            .sum()
            .to_frame()
            .reset_index())
    grup_km = grup_km[(grup_km['vendor_id'] == 'class_one')]
    grup_km["month_name"] = grup_km["month_name"].map({"JAN":1,"FEB":2,"MAR":3,
                                                    "APR":4,"MAY":5,"JUN":6})
    grup_km = grup_km.sort_values(by=['month_name'])
    grup_km["month_name"] = grup_km["month_name"].map({1:"JAN",2:"FEB",3:"MAR",
                                                    4:"APR",5:"MAY",6:"JUN"})
    round_km = round(grup_km['trip_distance_km'],0)
    grup_km['trip_distance_km'] = round_km


    def cont_pass_valeu(df):
        cont = go.Scatter(x=df['quant_pass'].map(str), 
                        y=df['passenger_count'],
                        mode = "lines+markers",
                        marker = dict(color = "royalblue",line = dict(width =1)) 
                        )
        return cont

    def grup_pickup_dropoff_week(day, category, df, pickup_dropoff_hour):
        nome = day
        
        tracer = go.Bar(x=df[(df['vendor_id'] == category) & (df['pickup_day_of_week'] == day)][pickup_dropoff_hour].map(str), 
                        y=df[(df['vendor_id'] == category) & (df['pickup_day_of_week'] == day)]['passenger_count'],
                        text=list(df[(df['vendor_id'] == category) & (df['pickup_day_of_week'] == day)]['passenger_count']), 
                        textposition='auto',
                        name = nome, marker = dict(line = dict(color = "black",
                                                                    width =  1.5))
                        )
        return tracer

    def grup_KM(month, distance, df):
        
        tracer = go.Bar(x=df[month], 
                        y=df[distance],
                        text=list(df[distance]),
                        textposition='auto',
                        marker = dict(line = dict(color = "black",
                                                width =  1.5))
                        )
        return tracer

    #layout
    def layout_plot(title, legend_title, xaxis, yaxis) :
        layout = go.Layout(dict(title = title,  title_x=0.5,
                            legend_title_text = legend_title,
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                        title = xaxis,
                                        zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                        title = yaxis,
                                        zerolinewidth=1,ticklen=5,gridwidth=2),
                                margin = dict(b = 100)
                            )
                        )
        return layout

    #figure 1
    data    = cont_pass_valeu(cont_pass)
    layout  = layout_plot('Total de passageiros por viagem categória de Táxi 1', None, 'Numéros de passageiros', 'Quantidade de viagens')
    fig = go.Figure(data = data,layout = layout)
    figure_one = py.iplot(fig)

    #figure 2
    data1 = [grup_pickup_dropoff_week('Monday', 'class_one', grup_pickup_week, 'pickup_hour'), grup_pickup_dropoff_week('Tuesday', 'class_one', grup_pickup_week, 'pickup_hour'), 
            grup_pickup_dropoff_week('Wednesday', 'class_one', grup_pickup_week, 'pickup_hour'), grup_pickup_dropoff_week('Thursday', 'class_one', grup_pickup_week, 'pickup_hour'),
            grup_pickup_dropoff_week('Friday', 'class_one', grup_pickup_week, 'pickup_hour'), grup_pickup_dropoff_week('Saturday', 'class_one', grup_pickup_week, 'pickup_hour'),
            grup_pickup_dropoff_week('Sunday', 'class_one', grup_pickup_week, 'pickup_hour')]
    layout1  = layout_plot('Total de passageiros por embarque categória de Táxi 1', 'Dias da Semana', 'Horário da corrida', 'Quantidade de passageiros')
    fig1 = go.Figure(data = data1,layout = layout1)
    figure_two = py.iplot(fig1)

    #figure 3
    data2 = [grup_pickup_dropoff_week('Monday', 'class_one', grup_dropoff_week, 'dropoff_hour'), grup_pickup_dropoff_week('Tuesday', 'class_one', grup_dropoff_week, 'dropoff_hour'), 
            grup_pickup_dropoff_week('Wednesday', 'class_one', grup_dropoff_week, 'dropoff_hour'), grup_pickup_dropoff_week('Thursday', 'class_one', grup_dropoff_week, 'dropoff_hour'),
            grup_pickup_dropoff_week('Friday', 'class_one', grup_dropoff_week, 'dropoff_hour'), grup_pickup_dropoff_week('Saturday', 'class_one', grup_dropoff_week, 'dropoff_hour'),
            grup_pickup_dropoff_week('Sunday', 'class_one', grup_dropoff_week, 'dropoff_hour')]
    layout2  = layout_plot('Total de passegeiros por desembarque categória de Táxi 1', 'Dias da Semana', 'Horário da corrida', 'Quantidade de passageiros')
    fig2 = go.Figure(data = data2,layout = layout2)
    figure_tree = py.iplot(fig2)

    #figure 5
    data4    = grup_KM('month_name', 'trip_distance_km', grup_km)
    layout4  = layout_plot('Total de quilômetros percoridos por meses categória de Táxi 1', None, 'Meses', 'Total de Quilômetros')
    fig4 = go.Figure(data = data4,layout = layout4)
    figure_five = py.iplot(fig4)

    return figure_one, figure_two, figure_tree, figure_five

In [ ]:
def analysis_two_co(df):

    JAN = ((df[(df['vendor_id'] == 'class_one') & (df['month_name'] == 'JAN') & (df['pickup_date'] >= '2016-01-01') & (df['pickup_date'] <= '2016-01-15')]['passenger_count']))
    FEB = ((df[(df['vendor_id'] == 'class_one') & (df['month_name'] == 'FEB') & (df['pickup_date'] >= '2016-02-01') & (df['pickup_date'] <= '2016-02-15')]['passenger_count']))
    MAR = ((df[(df['vendor_id'] == 'class_one') & (df['month_name'] == 'MAR') & (df['pickup_date'] >= '2016-03-01') & (df['pickup_date'] <= '2016-03-15')]['passenger_count']))
    APR = ((df[(df['vendor_id'] == 'class_one') & (df['month_name'] == 'APR') & (df['pickup_date'] >= '2016-04-01') & (df['pickup_date'] <= '2016-04-15')]['passenger_count']))
    MAY = ((df[(df['vendor_id'] == 'class_one') & (df['month_name'] == 'MAY') & (df['pickup_date'] >= '2016-05-01') & (df['pickup_date'] <= '2016-05-15')]['passenger_count']))
    JUN = ((df[(df['vendor_id'] == 'class_one') & (df['month_name'] == 'JUN') & (df['pickup_date'] >= '2016-06-01') & (df['pickup_date'] <= '2016-06-15')]['passenger_count']))

    x_data = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']

    y0 = JAN
    y1 = FEB
    y2 = MAR
    y3 = APR
    y4 = MAY
    y5 = JUN

    y_data = [y0, y1, y2, y3, y4, y5]

    colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
            'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']

    fig3 = go.Figure()

    for xd, yd, cls in zip(x_data, y_data, colors):
            fig3.add_trace(go.Box(
                y=yd,
                name=xd,
                boxpoints='all',
                jitter=0.5,
                whiskerwidth=0.2,
                fillcolor=cls,
                marker_size=2,
                line_width=1)
            )

    fig3 = fig3.update_layout(
        title='Distribuição de passageiros por meses com range de 0 a 15 dias categória de Táxi 1', title_x = 0.5,
        yaxis=dict(
            autorange=True,
            showgrid=True,
            zeroline=True,
            dtick=5,
            gridcolor='rgb(255, 255, 255)',
            gridwidth=1,
            zerolinecolor='rgb(255, 255, 255)',
            zerolinewidth=2,
        ),
        margin=dict(
            l=40,
            r=30,
            b=80,
            t=100,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=True
    )

    #figure 4
    figure_four = py.iplot(fig3)

    return figure_four

In [ ]:
def analysis_tree_co(df):

    grup_lat_lon_pic = (df.groupby(['vendor_id', 'month_name','pickup_latitude', 'pickup_longitude'])['passenger_count']
        .sum()
        .to_frame()
        .reset_index())
    
    fig5 = px.scatter_mapbox(grup_lat_lon_pic[(grup_lat_lon_pic['vendor_id'] == 'class_one')], lat="pickup_latitude", lon="pickup_longitude", color="month_name",
                        size="passenger_count", color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=11, zoom=8, mapbox_style="carto-positron")
    figure_five = fig5.update_layout(legend_title_text = "Meses", title = "Distribuição de passageiros no Embarque por categória de Táxi 1", title_x=0.5)

    return figure_five

In [ ]:
def analysis_four_co(df):

    grup_lat_lon_drop = (df.groupby(['vendor_id', 'month_name','dropoff_latitude', 'dropoff_longitude'])['passenger_count']
        .sum()
        .to_frame()
        .reset_index())
    
    fig6 = px.scatter_mapbox(grup_lat_lon_drop[(grup_lat_lon_drop['vendor_id'] == 'class_one')], lat="dropoff_latitude", lon="dropoff_longitude", color="month_name",
                        size="passenger_count", color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=11, zoom=8, mapbox_style="carto-positron")
    figure_six = fig6.update_layout(legend_title_text = "Meses", title = "Distribuição de passageiros no Desembarque por categória de Táxi 1", title_x=0.5)

    return figure_six